In [34]:
import pandas as pd
import numpy as np
import json

Numerical:
1. Likes
2. Comments

Categorical:
1. Tags, __DO NOT FORGET TO REMOVE TARGET TAGS OR SIMPLY DROP THIS COL__
2. Nickname, is this col meaningful?
3. Time gap

Text:
1. Title (PuncTitle)
2. Text (PuncText)

Time:
1. Timestamp

In [35]:
df = pd.read_csv("../data/preprocessed_strict.csv", index_col=0)

In [36]:
df.drop(columns=['CURRENT_TIMESTAMP'], inplace=True)
df.TITLE = df.TITLE.str.lower()
df["TIMESTAMP"] = pd.to_datetime(df['TIMESTAMP'])
print(df.shape)
df.drop(df[df['TITLE'] == 'удалено'].index, inplace=True)
df["TITLE"][df['TITLE'].isnull()] = ''
print(df.shape)

(51416, 11)
(51410, 11)


<ipython-input-36-15224741ae41>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TITLE"][df['TITLE'].isnull()] = ''


In [37]:
df["HOUR"] = df["TIMESTAMP"].map(lambda x: x.hour)
df["YEAR"] = df["TIMESTAMP"].map(lambda x: x.year)
df["MONTH"] = df["TIMESTAMP"].map(lambda x: x.month)
df["WEEKDAY"] = df["TIMESTAMP"].map(lambda x: x.weekday())
df["WEEK"] = df["TIMESTAMP"].map(lambda x: x.week)

<ipython-input-37-9e8a95b6654e>:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df["WEEK"] = df["TIMESTAMP"].map(lambda x: x.week)


In [38]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,TEXT_PUNCT,TITLE_PUNCT,target,HOUR,YEAR,MONTH,WEEKDAY,WEEK
0,хорошо быть бабой,245,\nхорошо быть бабой загрустила сфоткала себ...,"[""Девушки"", ""Счастье""]",38,Viruso4ek,5 лет назад,2014-12-22 23:52:56+03:00,"\nхорошо быть бабой - загрустила, сфоткала себ...",Хорошо быть бабой,0,23,2014,12,0,52
1,рим история циклична,3,\nлегионы легионы \nопоясали страну\nматерят ...,"[""Рим"", ""История"", ""Война"", ""Стихи""]",12,lokibrother,5 лет назад,2014-12-22 23:49:08+03:00,"\nлегионы, легионы \nопоясали страну\nматерят ...",Рим(история циклична),6,23,2014,12,0,52
2,языковой барьер,73,\n семья по долгу отцовской профессии по...,"[""Язык"", ""Украина"", ""Юмор""]",3,alekrasnodar,5 лет назад,2014-12-22 23:38:20+03:00,"\n семья, по долгу отцовской профессии, по...",Языковой барьер.,7,23,2014,12,0,52
3,не надо стесняться,79,\nа что плохого в онанизме в конце концов вы...,"[""Вуди Аллен"", ""Юмор""]",15,durdom0550,5 лет назад,2014-12-22 23:37:24+03:00,"\nа что плохого в онанизме? в конце концов, вы...",не надо стесняться,7,23,2014,12,0,52
4,не могу вспомнить название пластинки из детства,4,\nвсем привет в детстве была у меня пластинка...,"[""Пластинка"", ""Склероз"", ""Помощь"", ""Старое"", ""...",10,NiiBird,5 лет назад,2014-12-22 23:25:27+03:00,"\nвсем привет, в детстве была у меня пластинка...",Не могу вспомнить название пластинки из детства.,4,23,2014,12,0,52


In [39]:
df.drop(columns=["TAGS", "NICKNAME", "TEXT_PUNCT", "TITLE_PUNCT", "YEAR", "WEEK", "TIMESTAMP", "TIME_GAP"], inplace=True)

In [40]:
df.head()

,TITLE,LIKES,TEXT,COMMENTS,target,HOUR,MONTH,WEEKDAY
0,хорошо быть бабой,245,\nхорошо быть бабой загрустила сфоткала себ...,38,0,23,12,0
1,рим история циклична,3,\nлегионы легионы \nопоясали страну\nматерят ...,12,6,23,12,0
2,языковой барьер,73,\n семья по долгу отцовской профессии по...,3,7,23,12,0
3,не надо стесняться,79,\nа что плохого в онанизме в конце концов вы...,15,7,23,12,0
4,не могу вспомнить название пластинки из детства,4,\nвсем привет в детстве была у меня пластинка...,10,4,23,12,0


In [41]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

In [42]:
def make_cos_list(list_, period=24):
    def make_cos(value, period=period):
        return np.cos(value*2*np.pi/period)
    return [make_cos(x) for x in list_]

In [43]:
def make_sin_list(list_, period=24):
    def make_sin(value, period=period):
        return np.sin(value*2*np.pi/period)
    return [make_sin(x) for x in list_]

In [44]:
make_sin_list([1]), make_cos_list([1])

([0.25881904510252074], [0.9659258262890683])

In [45]:
df['sin_hour'] = make_sin_list(df['HOUR'])
df['cos_hour'] = make_cos_list(df['HOUR'])

df['cos_month'] = make_cos_list(df['MONTH'], 12)
df['sin_month'] = make_sin_list(df["MONTH"], 12)

df['cos_weekday'] = make_cos_list(df['WEEKDAY'], 7) 
df['sin_weekday'] = make_sin_list(df['WEEKDAY'], 7)

df.drop(columns=['HOUR', 'MONTH', 'WEEKDAY'], inplace=True)

In [46]:
df.drop(index = df.loc[~(df['TITLE'].str.len() > 0)].index, inplace=True)

In [47]:
df

,TITLE,LIKES,TEXT,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday
0,хорошо быть бабой,245,\nхорошо быть бабой загрустила сфоткала себ...,38,0,-0.258819,0.965926,1.0,-2.449294e-16,1.000000,0.000000
1,рим история циклична,3,\nлегионы легионы \nопоясали страну\nматерят ...,12,6,-0.258819,0.965926,1.0,-2.449294e-16,1.000000,0.000000
2,языковой барьер,73,\n семья по долгу отцовской профессии по...,3,7,-0.258819,0.965926,1.0,-2.449294e-16,1.000000,0.000000
3,не надо стесняться,79,\nа что плохого в онанизме в конце концов вы...,15,7,-0.258819,0.965926,1.0,-2.449294e-16,1.000000,0.000000
4,не могу вспомнить название пластинки из детства,4,\nвсем привет в детстве была у меня пластинка...,10,4,-0.258819,0.965926,1.0,-2.449294e-16,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
54595,вдруг откуда не возьмись,-70,\nеще совсем недавно все политологи и экспе...,7,3,0.258819,0.965926,-1.0,1.224647e-16,-0.900969,-0.433884
54596,совет будущим студентам,1771,\nсамое главное в общежитии сохранять хороши...,72,0,0.000000,1.000000,-1.0,1.224647e-16,-0.900969,-0.433884
54597,пара анекдотов,-12,\nк психоaнaлитику приходит нa прием женщинa ...,1,7,0.000000,1.000000,-1.0,1.224647e-16,-0.900969,-0.433884
54598,правдивая история о том как лена работала в п...,512,\nя с большим наслаждением читала рассказы пик...,80,1,0.000000,1.000000,-1.0,1.224647e-16,-0.900969,-0.433884


In [48]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/memesrized/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
from compress_fasttext.models import CompressedFastTextKeyedVectors

In [50]:
embs = "../embeddings/pruned_ft_500K.bin"
embeddings = CompressedFastTextKeyedVectors.load(str(embs))

In [51]:
def embed(tokens, default_size=100):
    if not tokens:
        return np.zeros(default_size)
    embs = [embeddings[_normalize_string(x)] for x in tokens]
    return sum(embs) / len(tokens)

def _normalize_string(text):
    return text.replace('ё', 'е')

def remove_stop_words(words_list):
    return [x for x in words_list if x not in sw]

In [52]:
def process_record(record):
    return embed(remove_stop_words(record.split()))

In [53]:
df['emb_title'] = df['TITLE'].map(process_record)

In [54]:
df['emb_text'] = df['TEXT'].map(str).map(process_record)

In [55]:
df.drop(columns=['TITLE', 'TEXT'], inplace=True)

In [56]:
df.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_title,emb_text
0,245,38,0,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,"[0.7117634392581887, 0.567797237206484, -0.508...","[0.148086874332409, 0.3802853105854503, -0.512..."
1,3,12,6,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,"[0.4857122787731308, 0.303521774739317, -0.641...","[0.0951560846172552, 0.07236642255955679, -0.2..."
2,73,3,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,"[-0.12102494323613686, -0.025923628939075863, ...","[0.15600298802570453, 0.11917518017961133, -0...."
3,79,15,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,"[0.14427872108140952, 0.6238722306915692, -0.2...","[0.2422986810867321, 0.06495182735686247, -0.3..."
4,4,10,4,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,"[0.2623649594252681, 0.3036175033559366, -0.25...","[0.09665790585936324, 0.21364064959749965, -0...."


In [57]:
df['emb_title'] = df['emb_title'].map(lambda x: json.dumps([float(y) for y in x]))
df['emb_text'] = df['emb_text'].map(lambda x: json.dumps([float(y) for y in x]))

In [58]:
df.to_csv("../data/final_serialized.csv")

In [59]:
df.isnull().sum()

LIKES          0
COMMENTS       0
target         0
sin_hour       0
cos_hour       0
cos_month      0
sin_month      0
cos_weekday    0
sin_weekday    0
emb_title      0
emb_text       0
dtype: int64

In [60]:
emb_text = pd.DataFrame(df['emb_text'].map(json.loads).to_list(),
                        columns=[f"emb_text_{i}" for i in range(100)])

emb_title = pd.DataFrame(np.array(df['emb_title'].map(json.loads).to_list()),
                         columns=[f"emb_title_{i}" for i in range(100)])

In [61]:
df_embedded = pd.concat([df.drop(columns=['emb_title', 'emb_text']).reset_index(drop=True),
                         emb_text, emb_title], axis=1)

In [62]:
df_embedded.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_text_0,...,emb_title_90,emb_title_91,emb_title_92,emb_title_93,emb_title_94,emb_title_95,emb_title_96,emb_title_97,emb_title_98,emb_title_99
0,245,38,0,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.148087,...,0.531877,-0.295999,0.432405,-0.171743,-0.376144,-0.080770,0.780307,-0.393801,0.284733,-0.529976
1,3,12,6,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.095156,...,0.207497,-0.087502,0.449081,0.798268,-0.418835,-0.091712,0.177536,-0.081389,-0.373750,-0.300206
2,73,3,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.156003,...,0.600254,-0.218309,0.488272,0.004122,-0.580864,-0.647737,-0.421757,0.404062,-0.599431,-0.755318
3,79,15,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.242299,...,0.178026,0.402201,0.071325,-0.381747,-0.243778,-0.708453,0.065975,-0.481236,-0.349474,-0.470402
4,4,10,4,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.096658,...,0.045307,0.060338,0.306918,-0.052441,0.083353,-0.437869,0.048654,-0.002049,-0.534763,-0.219268


In [63]:
df_embedded.to_csv("../data/final_embedded.csv")